In [33]:
from coffea.util import load
import numpy as np
import pandas as pd
import glob, os, json, argparse
years = ['2016preVFP', '2016postVFP', '2018','2017']
var_dict = [{}, {}, {}]
df = []
for year in years:
  print(f'Processing {year}')
  result = load(f"../results/{year}/makeDF/output_reso2.coffea")
  if isinstance(result,tuple):
      result = result[0]
  for varName in result:
    for i in range(3):
      if f'{i}jets' in varName:
        if varName.replace(f'_{i}jets','') in var_dict[i]:
          var_dict[i][varName.replace(f'_{i}jets','')] = np.append(var_dict[i][varName.replace(f'_{i}jets','')],result[varName].value)
        else:
          var_dict[i][varName.replace(f'_{i}jets','')] = result[varName].value
for i in range(3):
    df.append(pd.DataFrame(var_dict[i]))


Processing 2016preVFP
Processing 2016postVFP
Processing 2018
Processing 2017


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = [], [], [], []
for njet in [0,1,2]:
    data = df[njet]
    data_clean = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]
    data_cuts = data_clean[(data_clean['weight']!=0)&(data_clean['opp_charge']==1)&(data_clean['e_m_Mass']>115)&(data_clean['e_m_Mass']<135)&((data_clean['label']==1)|(data_clean['label']==0))]
    if njet==2:
        data_vbf = data_cuts[data_cuts['isVBFcat']==0]
    else:
        data_vbf = data_cuts

    # split data into X and y
    X = data_vbf
    Y = data_vbf['label']

    # split data into train and tsest sets
    seed = 123
    test_size = 0.30
    X_train_tmp, X_test_tmp, y_train_tmp, y_test_tmp = train_test_split(X, Y, test_size=test_size, random_state=seed)
    X_train.append(X_train_tmp)
    X_test.append(X_test_tmp)
    y_train.append(y_train_tmp)
    y_test.append(y_test_tmp)

In [35]:
import xgboost as xgb
BDTjsons = ['model_GG_0jets', 'model_GG_1jets', 'model_GG_2jets']
BDTmodels, BDTvars = {}, {}
for i, BDTjson in enumerate(BDTjsons):
    BDTmodels = xgb.XGBClassifier()
    BDTmodels.load_model(f'results/{BDTjson}.json')
    BDTvars = BDTmodels.get_booster().feature_names
    X_train[i]['mva'] = BDTmodels.predict_proba(X_train[i][BDTvars])[:,1]
    X_test[i]['mva'] = BDTmodels.predict_proba(X_test[i][BDTvars])[:,1]

/tmp/ipykernel_4172868/2047137999.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[i]['mva'] = BDTmodels.predict_proba(X_train[i][BDTvars])[:,1]
/tmp/ipykernel_4172868/2047137999.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[i]['mva'] = BDTmodels.predict_proba(X_test[i][BDTvars])[:,1]


In [36]:
from my_roc_auc3 import my_roc_auc
for i in range(3):
    print('Train', my_roc_auc(y_train[i].to_numpy(), X_train[i]['mva'].to_numpy(), X_train[i]['weight'].to_numpy()))
    print('Test', my_roc_auc(y_test[i].to_numpy(), X_test[i]['mva'].to_numpy(), X_test[i]['weight'].to_numpy()))

Train 0.8269621956239436
Test 0.8332518177986797
Train 0.8452356291331247
Test 0.8494580284382419
Train 0.9017741287934311
Test 0.8982671258249053


In [37]:
X_train_mva = np.append(np.append(X_train[0]['mva'].to_numpy(), X_train[1]['mva'].to_numpy()), X_train[2]['mva'].to_numpy())
X_test_mva = np.append(np.append(X_test[0]['mva'].to_numpy(), X_test[1]['mva'].to_numpy()), X_test[2]['mva'].to_numpy())
y_train_ = np.append(np.append(y_train[0].to_numpy(), y_train[1].to_numpy()), y_train[2].to_numpy())
y_test_ = np.append(np.append(y_test[0].to_numpy(), y_test[1].to_numpy()), y_test[2].to_numpy())
X_train_weight = np.append(np.append(X_train[0]['weight'].to_numpy(), X_train[1]['weight'].to_numpy()), X_train[2]['weight'].to_numpy())
X_test_weight = np.append(np.append(X_test[0]['weight'].to_numpy(), X_test[1]['weight'].to_numpy()), X_test[2]['weight'].to_numpy())
print('Train', my_roc_auc(y_train_, X_train_mva, X_train_weight))
print('Test', my_roc_auc(y_test_, X_test_mva, X_test_weight))

Train 0.8443404751607844
Test 0.8480394950550113
